In [11]:
from narrative_llm_agent.crews.job_crew import JobCrew
from narrative_llm_agent.crews.startup_crew import StartupCrew
from langchain_openai import ChatOpenAI
import os
from crewai import Crew, Agent, Task
from narrative_llm_agent.agents.workflow import WorkflowRunner
from narrative_llm_agent.agents.job import AppOutputInfo
from langchain.tools import tool
import json
from pydantic import BaseModel, Field, PrivateAttr
from typing import Type

# Combine the intro (get object and metadata) and app running workflows.
Should do the following, in order.
1. X User indepedently makes a new narrative and adds a reads object or two to it.
2. X User notes narrative id. (possible we can have the agent fetch the most recent few, start with narrative id)
3. X Run the startup crew
4. X Should return object with narrative id, reads, metadata
5. (later) Run the Analyst, set up apps to run (for now, use curated set from Prachi)
6. Run Workflow Runner over them all.

In [2]:
narrative_id = 211157

In [3]:
llm = ChatOpenAI(
    model="openai/gpt-4o",
    api_key=os.environ.get("CBORG_API_KEY"),
    base_url="https://api.cborg.lbl.gov",  # For LBL-Net, use "https://api-local.cborg.lbl.gov"
    temperature=0,
)

In [4]:
startup_crew = StartupCrew(llm)

In [5]:
result = startup_crew.run_startup()

# Agent: Project coordinator
## Task: 
            Ask the user which narrative id they are using. This will be a number. Return only that number for the next task.
        
# Agent: Human Interaction Manager
## Task: Ask the user which narrative id they are using. This will be a number. Return only that number for the next task.
Could you please provide the narrative id you are using? It should be a number.


 211157




# Agent: Human Interaction Manager
## Thought: Thought: I need to ask the user for the narrative id they are using, which is a number, to proceed with the next task.
## Using tool: conversation-tool
## Tool Input: 
"{\"argument\": \"Could you please provide the narrative id you are using? It should be a number.\"}"
## Tool Output: 
211157


# Agent: Human Interaction Manager
## Final Answer: 
211157
```




# Agent: Project coordinator
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Ask the user which narrative id they are using. This will be a number. Return only that number for the next task.\", \"context\": \"We need to obtain the numeric narrative id from the user to proceed with the next task. This id is crucial for the project's workflow.\", \"coworker\": \"Human Interaction Manager\"}"
## Tool Output: 
211157
```


# Agent: Project coordinator
## Final Answer: 
211157
```


# Agent: Workspace Manager
## Task: 
            Fetch the list of all reads obje

 Yes, use the only one available.




# Agent: Human Interaction Manager
## Thought: I need to ask the user which set of reads they want to assemble and annotate from the available list.
## Using tool: conversation-tool
## Tool Input: 
"{\"argument\": \"Which set of reads would you like to assemble and annotate? The available option is 'rhodo.art.q20.PE.reads'.\"}"
## Tool Output: 
Yes, use the only one available.


# Agent: Human Interaction Manager
## Final Answer: 
The user has chosen to assemble and annotate the reads set 'rhodo.art.q20.PE.reads'.




# Agent: Project coordinator
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Ask the user which set of reads they want to assemble and annotate from the available list.\", \"context\": \"The available reads are: [{'narrative_id': 211157, 'reads_object_name': 'rhodo.art.q20.PE.reads', 'reads_object_upa': '211157/2/1'}]. We need to know the user's choice to proceed with the assembly and annotation.\", \"coworker\": \"Human Interaction Manager\"}"
##

...



# Agent: Human Interaction Manager
## Using tool: conversation-tool
## Tool Input: 
"{\"argument\": \"Hello! I need to gather some information about your genomic reads to proceed with the assembly and annotation. Could you please provide details on the organism, read length, insert size, library type, whether the reads are paired-end, sequencing platform, sequencing depth, genome size, ploidy, and coverage? Additionally, if you have any preferred assembler, annotator, or QC tools, please let me know. If you're unsure about any of these, that's perfectly fine too!\"}"
## Tool Output: 
The reads are paired-end, and they come from a Rhodobacter species.
Great, thank you! Could you also provide the read length, insert size, library type, sequencing platform, sequencing depth, genome size, ploidy, and coverage? Additionally, if you have any preferred assembler, annotator, or QC tools, please let me know. If you're unsure about any of these, that's perfectly fine too!


 I have no preference.




# Agent: Human Interaction Manager
## Using tool: conversation-tool
## Tool Input: 
"{\"argument\": \"Great, thank you! Could you also provide the read length, insert size, library type, sequencing platform, sequencing depth, genome size, ploidy, and coverage? Additionally, if you have any preferred assembler, annotator, or QC tools, please let me know. If you're unsure about any of these, that's perfectly fine too!\"}"
## Tool Output: 
I have no preference.


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: conversation-tool
Tool Arguments: {'argument': {'description': None, 'type': 'str'}}
Tool Description: Converse with the user until all questions are fully answered.
Tool Name: get-object-metadata
Tool Arguments: {'obj_upa': {'description': None, 'type': 'str'}}
Tool Description: Return the metadata for a KBase Workspace object with the given UPA.
Tool Name: store-conversation
Tool Arguments: {'narrative_id': {'desc

 I don't know any of those.




# Agent: Human Interaction Manager
## Using tool: conversation-tool
## Tool Input: 
"{\"argument\": \"Thank you for your patience. Could you please provide the read length, insert size, library type, sequencing platform, sequencing depth, genome size, ploidy, and coverage? If you're unsure about any of these, that's perfectly fine too!\"}"
## Tool Output: 
I don't know any of those.


# Agent: Human Interaction Manager
## Final Answer: 
{
  "organism": "Rhodobacter species",
  "read_length": null,
  "insert_size": null,
  "library_type": null,
  "paired_end": true,
  "platform": null,
  "sequencing_depth": null,
  "genome_size": null,
  "ploidy": null,
  "coverage": null,
  "additional_notes": null,
  "preferred_assembler": null,
  "preferred_annotator": null,
  "preferred_qc_tools": null
}
```




In [19]:
print(json.dumps(result.dict(), indent=4))

{
    "metadata": {
        "organism": "Rhodobacter species",
        "read_length": null,
        "insert_size": null,
        "library_type": null,
        "paired_end": true,
        "platform": null,
        "sequencing_depth": null,
        "genome_size": null,
        "ploidy": null,
        "coverage": null,
        "additional_notes": null,
        "preferred_assembler": null,
        "preferred_annotator": null,
        "preferred_qc_tools": null
    },
    "narrative_id": 211157,
    "reads_object_name": "rhodo.art.q20.PE.reads",
    "reads_object_upa": "211157/2/1"
}


In [14]:
wf_runner = WorkflowRunner(llm)

In [15]:
steps_to_run = [
    {
        "Step": 1,
        "Name": "Quality control",
        "Description": "Check reads for quality using FastQC",
        "expect_new_object": False,
        "app_id": "kb_fastqc/runFastQC"
    }, 
    {
        "Step": 2,
        "Name": "Reads trimming",
        "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic",
        "expect_new_object": True,
        "app_id": "kb_trimmomatic/run_trimmomatic"
    },
    {
        "Step": 3,
        "Name": "Genome Assembly",
        "Description": "Perform de novo assembly of the trimmed reads into contigs and scaffolds using SPAdes.",
        "expect_new_object": True,
        "app_id": "kb_SPAdes/run_SPAdes"
    },
    {
        "Step": 4,
        "Name": "Assembly Quality Assessment",
        "Description": "Obtain assembly statistics such as N50, total length, and number of contigs using QUAST.", 
        "expect_new_object": False,
        "app_id": "kb_quast/run_QUAST_app"
    },
    {
        "Step": 5, 
        "Name": "Genome Annotation", 
        "Description": "Annotate the assembled genome using Prokka to provide insights into gene content and functional elements.", 
        "expect_new_object": True,
        "app_id": "ProkkaAnnotation/annotate_contigs"
    },
    {
        "Step": 6, 
        "Name": "Genome Completeness Estimation", 
        "Description": "Estimate genome completeness and contamination levels using CheckM.", 
        "expect_new_object": False,
        "app_id": "kb_Msuite/run_checkM_lineage_wf"
    },
    {
        "Step": 7,
        "Name": "Build Genome Set",
        "Description": "Build a Genome Set to run through GTDB-tk",
        "expect_new_object": True,
        "app_id": "kb_SetUtilities/KButil_Build_GenomeSet",
    },
    {
        "Step": 8, 
        "Name": "Taxonomic Classification", 
        "Description": "Perform taxonomic classification to confirm organism identity using GTDB-Tk.", 
        "app_id": "kb_gtdbtk/run_kb_gtdbtk_classify_wf",
        "expect_new_object": True
    }
]

In [16]:
run_apps_task = Task(
    description=f"""
    This task involves running multiple apps where the output of one (if any) is fed into the next as input. 
    Here are the tasks in JSON format: {json.dumps(steps_to_run)}.
    If any task has "expect_new_object" set to True, then that should receive a new data object in its output as a "created_object". That object should be used as input for the next task.
    If a task as "expect_new_object" set to False, then that should not receive a new object to use in the next task. In that case, use the same input object from the previous step for the next one.
    These steps must be run sequentially. 
    These must be run in the narrative with id {result.narrative_id} and start with using the paired-end reads object {result.reads_object_upa} with name {result.reads_object_name}.
    If any step ends with an error, immediately stop the task and end with an error.
    In the end, return a brief summary of steps taken and resulting output objects.
    """,
    expected_output="A summary of task completion, the number of apps run, and the upa of any output objects.",
    agent=wf_runner.agent
)

In [17]:
crew = Crew(
    agents = [wf_runner.agent],
    tasks=[run_apps_task],
    verbose=True,
)
crew.kickoff()

Overriding of current TracerProvider is not allowed


# Agent: You are a workflow runner, your role is to efficiently run KBase workflows.
## Task: 
    This task involves running multiple apps where the output of one (if any) is fed into the next as input. 
    Here are the tasks in JSON format: [{"Step": 1, "Name": "Quality control", "Description": "Check reads for quality using FastQC", "expect_new_object": false, "app_id": "kb_fastqc/runFastQC"}, {"Step": 2, "Name": "Reads trimming", "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic", "expect_new_object": true, "app_id": "kb_trimmomatic/run_trimmomatic"}, {"Step": 3, "Name": "Genome Assembly", "Description": "Perform de novo assembly of the trimmed reads into contigs and scaffolds using SPAdes.", "expect_new_object": true, "app_id": "kb_SPAdes/run_SPAdes"}, {"Step": 4, "Name": "Assembly Quality Assessment", "Description": "Obtain assembly statistics such as N50, total length, and number of contigs using QUAST.", "expect_new

Overriding of current TracerProvider is not allowed


# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_fastqc/runFastQC, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "211157/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its UPA. If the input object name is not available, the Workspace Manager can assist.
            If the parameter type is 'dropdown', use the allowed 'name' option to determine

Overriding of current TracerProvider is not allowed


...

Overriding of current TracerProvider is not allowed


# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_SetUtilities/KButil_Build_GenomeSet, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "211157/9/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its UPA. If the input object name is not available, the Workspace Manager can assist.
            If the parameter type is 'dropdown', use the allowed 'name' 

Overriding of current TracerProvider is not allowed


# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_gtdbtk/run_kb_gtdbtk_classify_wf, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "211157/12/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its UPA. If the input object name is not available, the Workspace Manager can assist.
            If the parameter type is 'dropdown', use the allowed 'name' op

CrewOutput(raw='The workflow was successfully completed with the following steps and resulting output objects:\n\n1. Quality control using FastQC: Completed, no new object created.\n2. Reads trimming using Trimmomatic: Completed, created object "rhodo_art_q20_PE_reads_trimmed_paired" with UPA "211157/4/1".\n3. Genome Assembly using SPAdes: Completed, created object "rhodo_art_q20_PE_reads_trimmed_paired_Assembly" with UPA "211157/6/1".\n4. Assembly Quality Assessment using QUAST: Completed, no new object created.\n5. Genome Annotation using Prokka: Completed, created object "rhodo_art_q20_PE_reads_trimmed_paired_Assembly_Annotated" with UPA "211157/9/1".\n6. Genome Completeness Estimation using CheckM: Completed, no new object created.\n7. Build Genome Set using SetUtilities: Completed, created object "rhodo_art_q20_PE_reads_trimmed_paired_Assembly_Annotated_GenomeSet" with UPA "211157/12/1".\n8. Taxonomic Classification using GTDB-Tk: Completed, created objects "rhodo_art_q20_PE_reads